# Livedoorニュースコーパスを用いたニューストピック分類

### タスクの説明
LiveDoorニュースコーパスを用いて、ニュース記事のトピック分類を行います。  
本資料ではすでに加工済みのデータを利用しています。  
元データのダウンドードは[こちらから](https://www.rondhuit.com/download.html)行えます。  

このコーパスは各記事がどのトピックに属するかがラベル付けされています。
このコーパスに含まれるトピックは以下の通りです。
- トピックニュース
- Sports Watch
- ITライフハック
- 家電チャンネル
- MOVIE ENTER
- 独女通信
- エスマックス
- livedoor HOMME
- Peachy

今回実装するタスクは、ニュース記事の本文から上記の九つのトピックのうちどれに属するかを予測するマルチクラス分類です。

In [ ]:
# GPUが複数台搭載されている実行環境の場合、以下の環境変数を設定することで、使用するGPUを指定できます
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [ ]:
BASE_PATH = '/content/'

: 

In [ ]:
# 必要なライブラリのインストール
# NOTE: 仮想環境が適切に構築されていることを事前に確認して下さい
!pip install pandas transformers torch fugashi ipadic

In [ ]:
# seed値の設定
from transformers import set_seed
seed = 42
set_seed(seed)

### Livedoorニュースコーパスの読み込み

In [ ]:
with open(BASE_PATH+"categories.json","r") as f:
    d = f.read()
    import json
    categories = json.loads(d)
num_labels = len(categories)
print(categories)
print(f"num_labels: {num_labels}")

### データセットの準備

In [ ]:
import pandas as pd
df_train = pd.read_csv(BASE_PATH+"train.csv")
df_valid = pd.read_csv(BASE_PATH+"valid.csv")
df_test = pd.read_csv(BASE_PATH+"test.csv")



In [ ]:
print(df_train)

In [ ]:
# tokenizerの読み込み
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

# モデルの最大入力トークン数
max_length = tokenizer.model_max_length
print(f"model input max_length: {max_length}")

In [ ]:
# 今回用意されたcsvの形式が読み込まれたdata frameからtokenize後のデータセットを作成する関数を定義
import random
def create_dataset(df):
    dataset = []
    for i in range(len(df)):
        label = df.iloc[i, 0]
        text = df.iloc[i, 4]
        if label == "9" & random.randint(1, 10) > 1:
            continue
        dataset.append({
            "label": label,
            "text": text,
            "encoding": tokenizer(text, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
        })
    return dataset

In [ ]:
# 読み込んだデータフレームから１レコードが辞書型として格納されている配列を作成
print("creating train dataset...(this may take a while)")
train_dataset = create_dataset(df_train)
print("creating valid dataset...")
valid_dataset = create_dataset(df_valid)
print("creating test dataset...")
test_dataset = create_dataset(df_test)

In [ ]:
# データセットを確認
print(f"Number of samples: {len(train_dataset)}")
print(f"Sample 0: {train_dataset[0]}")

In [ ]:
# Dataloaderの作成
from torch.utils.data import DataLoader

batch_size = 64  # Hyperparameter

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=1, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(valid_dataset)}")
print(f"Number of test samples: {len(test_dataset)}")


### モデルの定義

In [ ]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', num_labels=num_labels)
# モデルの確認
print(model.classifier)

In [ ]:
import torch

lr = 1e-5 # Hyperparameter
optimizer_parameters = model.parameters()
# optimizerの設定(Adam)
optimizer = torch.optim.Adam(optimizer_parameters, lr=lr)

In [ ]:
# 学習を行うデバイスを設定
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(f"device: {device}")

In [ ]:
from tqdm import tqdm

losses = []

epoch_num = 3 # Hyperparameter

step_num = 1
epochs = 1

last_loss = float('inf')
loss_threshold = 0.02 # Hyperparameter

valid_step_period = len(train_dataloader) // 5

model.train()
model.to(device)

print("train start!!!")
for _ in range(epoch_num):
    print(f'########### Epoch {epochs} ###########')
    # train step
    for data in train_dataloader:
        # optimizerの初期化
        optimizer.zero_grad()

        input_ids = data["encoding"]["input_ids"].squeeze(1).to(device)
        attention_mask = data["encoding"]["attention_mask"].squeeze(1).to(device)
        token_type_ids = data["encoding"]["token_type_ids"].squeeze(1).to(device)
        labels = data["label"].to(device)
    
        model_output = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)
        logit = model_output.logits
        loss = model_output.loss
        output_label = torch.argmax(logit, dim=1)
    
        losses.append(loss.item())
    
        loss.backward()
        optimizer.step()
        last_loss = loss.item()
        step_num += 1
    
        # validation step
        if step_num % valid_step_period == 0:
            valid_output_labels = []
            valid_correct_labels = []
            model.eval()
            for data in valid_dataloader:
                labels = data["label"].to(device)
                
                model_output = model(**{k: v.squeeze(1).to(device) for k, v in data["encoding"].items()}, labels=labels)
                # モデルの最終層の出力
                logit = model_output.logits
                # モデルの最終層の出力が最も高いラベルを予測ラベルとする
                output_label = torch.argmax(logit, dim=1)

                valid_output_labels.append(output_label)
                valid_correct_labels.append(labels)

            # accuracyの算出：モデルの出力ラベルと正解ラベルが一致している数を全データ数で割った値を算出
            valid_accuracy = torch.sum(torch.cat(valid_output_labels) == torch.cat(valid_correct_labels)).item() / len(valid_dataset)
            print(f"Step {step_num} / {len(train_dataloader)}, Last Loss {last_loss}, Valid Accuracy {valid_accuracy}")
            # モデルを学習モードに戻す
            model.train()

        # early stopping
        if last_loss < loss_threshold:
            print("Loss is less than threshold!")
            break
    epochs += 1
    step = 0
print("train finish!!!")

In [ ]:
# test step
test_correct_num = 0
test_total_num = len(test_dataset)

model.eval()
for data in tqdm(test_dataloader):
    labels = data["label"].to(device)

    model_output = model(**{k: v.squeeze().to(device) for k,v in data["encoding"].items()}, labels=labels)
    logit = model_output.logits
    output_label = torch.argmax(logit, dim=1)

    test_correct_num += torch.sum(output_label == labels).item()

In [ ]:
accuracy = test_correct_num / test_total_num
print(f"correct samples | total samples : {test_correct_num} | {test_total_num}")
print(f"Accuracy: {accuracy}")

In [ ]:
# モデルの保存
model_dir = BASE_PATH+"model/"
model_full_path = model_dir + "model.pth"
os.makedirs(model_dir, exist_ok=True)
torch.save(model.state_dict(), model_full_path)